# Wide ResNet
From: https://github.com/meliketoy/wide-resnet.pytorch/blob/master/networks/wide_resnet.py

Just altered the training script to take different save dir

In [1]:
import os
import sys
import random
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from IPython.display import display, HTML

import scipy.io

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import torch
import torch.nn as nn
import torch.utils.data
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50, densenet121
from torchvision import datasets, models, transforms


%reload_ext autoreload
%autoreload 2
%matplotlib inline

PATH = Path('/media/rene/data')

# Add the src directory for functions
src_dir = Path.cwd().parent.parent / 'src'
print(src_dir)
sys.path.append(str(src_dir))

# import my functions:
from utils import make_cfiar10
from functions import*
from models import*

print(torch.cuda.device_count())
print(torch.cuda.is_available())
print(torch.cuda.current_device())

/media/rene/code/idc/src
2
True
0
